<a href="https://colab.research.google.com/github/JGH94/Colab_ML/blob/main/CNN_TensorBoard.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import tensorflow as tf

In [2]:
(X_Train,Y_Train),(X_Test,Y_Test) = tf.keras.datasets.mnist.load_data()
X_Train,X_Test = X_Train.astype('float32'), X_Test.astype('float32')
X_Train, X_Test = X_Train.reshape([-1,784]), X_Test.reshape([-1, 784])
X_Train, X_Test = X_Train/255., X_Test/255.
Y_Train, Y_Test = tf.one_hot(Y_Train, depth= 10), tf.one_hot(Y_Test, depth= 10)

11501568/11490434 [==============================] - 0s 0us/step


In [3]:
Train_Data= tf.data.Dataset.from_tensor_slices((X_Train,Y_Train))
Train_Data= Train_Data.repeat().shuffle(60000).batch(50)
Train_Data_Iter = iter(Train_Data)

In [4]:
class CNN(tf.keras.Model):
  def __init__(self):
    super(CNN, self).__init__()
    # Convolution layer 
    # 5 X 5 kernel Size / 32개 Filter
    self.Conv_Layer_1 = tf.keras.layers.Conv2D(filters=32, kernel_size=5, strides=1, padding='same', activation= 'relu')
    self.Pool_Layer_1 = tf.keras.layers.MaxPool2D(pool_size=(2,2), strides =2)

    # Convolustion Layer 2
    # 5 X 5 kernel Size / 64개 Filter
    self.Conv_Layer_2 = tf.keras.layers.Conv2D(filters=64, kernel_size=5, strides=1, padding='same', activation= 'relu')
    self.Pool_Layer_2 = tf.keras.layers.MaxPool2D(pool_size=(2,2), strides =2)

    # Fully Connected Layer
    # 7 X 7 크기 64개의 activation map 1024개로 변환
    self.Flatten_Layer = tf.keras.layers.Flatten()
    self.FC_Layer_1 = tf.keras.layers.Dense(1024, activation='relu')

    # Output Layer
    # 1024개 특징을 10개의 클래스 one-hot-encoding으로 표현된 숫자로 변환
    self.Output_Layer = tf.keras.layers.Dense(10, activation=None)
  
  def call(self, x):
    #MNIST 데이터를 3차원 형태로 reshape
    X_Image = tf.reshape(x, [-1, 28, 28, 1])
    # 28X28X1을 28X28X32로 변환, -> 14X14X32
    Conv_1 = self.Conv_Layer_1(X_Image)
    Pool_1 = self.Pool_Layer_1(Conv_1)
    # 14X14X32를 14X14X64 변환, -> 7X7X64
    Conv_2 = self.Conv_Layer_2(Pool_1)
    Pool_2 = self.Pool_Layer_2(Conv_2)

    #7X7X64 -> 1024
    Pool_2_Flat = self.Flatten_Layer(Pool_2)
    FC_1 = self.FC_Layer_1(Pool_2_Flat)

    # 1024 -> 10
    Logits = self.Output_Layer(FC_1)
    Y_Pred = tf.nn.softmax(Logits)
    return Y_Pred, Logits


In [5]:
def Cross_Entropy_Loss(Logits, Y):
  return tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(logits=Logits, labels=Y))


In [6]:
#최적화 Adam
optimizer = tf.optimizers.Adam(1e-4)

In [7]:
# TensorBoard Summary 정보/ 저장할 폴더 경로 설정 및 FileWriter 
Train_Summary_Write = tf.summary.create_file_writer('./TensorBoard/Train')
Test_Summary_Write = tf.summary.create_file_writer('./TensorBoard/Test')

In [8]:
def train_step(model, x, y):
  with tf.GradientTape() as tape:
    y_pred, logits = model(x)
    loss = Cross_Entropy_Loss(logits, y)
  # 매 step마다 tf.summary.scalar, tf.summary.image 텐서보드 로그를 기록합니다.
  with Train_Summary_Write.as_default():
    tf.summary.scalar('loss', loss, step=optimizer.iterations)
    x_image = tf.reshape(x, [-1, 28, 28, 1])
    tf.summary.image('training image', x_image, max_outputs=10, step=optimizer.iterations) 
  gradients = tape.gradient(loss, model.trainable_variables)
  optimizer.apply_gradients(zip(gradients, model.trainable_variables))


In [11]:
def Compute_Accuracy(Y_Pred, Y, Summary_writer):
  correct_pr = tf.equal(tf.argmax(Y_Pred,1), tf.argmax(Y,1))
  accuracy = tf.reduce_mean(tf.cast(correct_pr, tf.float32))

  with Summary_writer.as_default():
    tf.summary.scalar('accuracy',accuracy, step= optimizer.iterations)
  return accuracy
CNN_Model = CNN()

In [14]:
save_path ="./SaveModel"
Training_CK = tf.train.Checkpoint(step=tf.Variable(0), model = CNN_Model) 
Training_CK_Manager = tf.train.CheckpointManager(Training_CK, directory= save_path, max_to_keep= 5)
Training_Last = tf.train.latest_checkpoint(save_path)


In [16]:
# 저장파일이 존재하면 불러와서 Restored하여 학습진행
if Training_Last:
  Training_CK.restore(Training_Last)
  print("정확도 : %f"%Compute_Accuracy(CNN_Model(X_Test[0], Y_Test)))


In [17]:
while int(Training_CK.step) <(1000 + 1):
  batch_x, batch_y = next(Train_Data_Iter)
  if Training_CK.step % 50 == 0:
    Training_CK_Manager.save(checkpoint_number = Training_CK.step)
    train_accuracy = Compute_Accuracy(CNN_Model(batch_x)[0], batch_y, Train_Summary_Write)
    print("Epoch: %d, Train data 정확도: %f" % (Training_CK.step, train_accuracy))
  
  train_step(CNN_Model, batch_x,batch_y)
  Training_CK.step.assign_add(1)


Epoch: 0, Train data 정확도: 0.020000
Epoch: 50, Train data 정확도: 0.760000
Epoch: 100, Train data 정확도: 0.840000
Epoch: 150, Train data 정확도: 0.920000
Epoch: 200, Train data 정확도: 0.940000
Epoch: 250, Train data 정확도: 0.940000
Epoch: 300, Train data 정확도: 0.940000
Epoch: 350, Train data 정확도: 0.980000
Epoch: 400, Train data 정확도: 0.920000
Epoch: 450, Train data 정확도: 0.960000
Epoch: 500, Train data 정확도: 0.980000
Epoch: 550, Train data 정확도: 0.920000
Epoch: 600, Train data 정확도: 0.960000
Epoch: 650, Train data 정확도: 0.960000
Epoch: 700, Train data 정확도: 0.960000
Epoch: 750, Train data 정확도: 1.000000
Epoch: 800, Train data 정확도: 0.960000
Epoch: 850, Train data 정확도: 0.920000
Epoch: 900, Train data 정확도: 0.940000
Epoch: 950, Train data 정확도: 0.980000
Epoch: 1000, Train data 정확도: 0.920000


In [ ]:

#for i in range(1000):
  #batch_x, batch_y = next(Train_Data_Iter)
  #if i % 50 == 0:
    #train_accuracy = Compute_Accuracy(CNN_Model(batch_x)[0], batch_y, Train_Summary_Write)
    #print("Epoch: %d, Train data 정확도: %f" % (i, train_accuracy))
  #train_step(CNN_Model, batch_x,batch_y) 
#print("정확도(Accuracy): %f" % Compute_Accuracy(CNN_Model(X_Test)[0], Y_Test, Test_Summary_Write))

Epoch: 0, Train data 정확도: 0.120000
Epoch: 50, Train data 정확도: 0.680000
Epoch: 100, Train data 정확도: 0.880000
Epoch: 150, Train data 정확도: 0.880000
Epoch: 200, Train data 정확도: 0.940000
Epoch: 250, Train data 정확도: 0.960000
Epoch: 300, Train data 정확도: 0.960000
Epoch: 350, Train data 정확도: 0.900000
Epoch: 400, Train data 정확도: 0.960000
Epoch: 450, Train data 정확도: 1.000000
Epoch: 500, Train data 정확도: 0.920000
Epoch: 550, Train data 정확도: 0.960000
Epoch: 600, Train data 정확도: 0.900000
Epoch: 650, Train data 정확도: 0.980000
Epoch: 700, Train data 정확도: 0.940000
Epoch: 750, Train data 정확도: 0.980000
Epoch: 800, Train data 정확도: 1.000000
Epoch: 850, Train data 정확도: 0.920000
Epoch: 900, Train data 정확도: 0.960000
Epoch: 950, Train data 정확도: 0.980000
정확도(Accuracy): 0.973400
